In [ ]:
!pip install mediapipe
!pip install flask flask-ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip ngrok-v3-stable-linux-amd64.zip
!chmod +x ngrok
!./ngrok authtoken 2qNvyNh2aykr2PAYtriDS4BEOWm_54bmAjo1YsichvLq38to5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 53.6 MB/s eta 0:00:00
--2024-12-19 04:38:46--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 35.71.179.82, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918622 (14M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.zip’

ngrok-v3-stable-lin 100%[===================>]  14.23M  23.7MB/s    in 0.6s    

2024-12-19 04:38:47 (23.7 MB/s) - ‘ngrok-v3-stable-linux-amd64.zip’ saved [14918622/14918622]

Archive:  ngrok-v3-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import datetime

# 모델 불러오기
model = load_model('/content/best_model.keras')

# Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp.solutions.pose.Pose(
        static_image_mode=False,        # Real-time processing
        model_complexity=2,
        smooth_landmarks=False,         # Smooth the pose landmarks
        enable_segmentation=False,     # Disable segmentation if not needed
        min_detection_confidence=0.85,  # Minimum confidence for detection
        min_tracking_confidence=0.85    # Minimum confidence for tracking
    )
mp_drawing = mp.solutions.drawing_utils

# 클래스 매핑
class_mapping = {
    0: "stand",
    1: "sit",
    2: "walk",
    3: "bow",
    4: "fall"
}

# 주요 Keypoint 인덱스
KEYPOINTS_OF_INTEREST = [
    11, 12,  # Shoulders
    13, 14, 15, 16,
    23, 24,  # Hips
    25, 26,  # Knees
    27, 28   # Ankles
]

# 스켈레톤용 키포인트 묶음
KEYPOINTS_OF_INTEREST_PAIRS = [
    (0, 11), (0, 12),
    (11, 12), (12, 24), (23, 24), (11, 23),
    (11, 13), (13, 15),
    (12, 14), (14, 16),
    (23, 25), (25, 27),
    (24, 26), (26, 28)
]

# 주요 Keypoint를 추출하고 모델을 통해 분류하는 함수
def extract_keypoints(frame):
    # 프레임을 RGB로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe Pose 모델로 키포인트 추출
    results = pose.process(frame_rgb)

    # Keypoints가 존재하는지 확인
    if results.pose_landmarks:
        keypoints = []
        for i in KEYPOINTS_OF_INTEREST:
            landmark = results.pose_landmarks.landmark[i]
            keypoints.extend([landmark.x, landmark.y, landmark.z])
        return np.array(keypoints), results.pose_landmarks
    return None, None

# 분류된 결과를 텍스트로 변환하는 함수
def classify_motion(keypoints):
    keypoints = keypoints.reshape(1, -1)  # 모델에 입력할 형태로 변경
    predictions = model.predict(keypoints)  # 모델 예측
    class_id = np.argmax(predictions, axis=1)[0]  # 가장 높은 확률의 클래스 선택
    return class_mapping[class_id]

# 주요 Keypoint를 그리는 함수
def draw_keypoints(frame, keypoints):
    for i in range(0, len(KEYPOINTS_OF_INTEREST) * 3, 3):
        x, y = int(keypoints[i] * frame.shape[1]), int(keypoints[i + 1] * frame.shape[0])
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

# 스켈레톤을 그리는 함수
def draw_skeleton(frame, landmarks):
    for pair in KEYPOINTS_OF_INTEREST_PAIRS:
        start_idx, end_idx = pair
        start = landmarks.landmark[start_idx]
        end = landmarks.landmark[end_idx]
        start_point = (int(start.x * frame.shape[1]), int(start.y * frame.shape[0]))
        end_point = (int(end.x * frame.shape[1]), int(end.y * frame.shape[0]))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)

# 분류된 결과와 클래스별 근사도를 텍스트로 변환하는 함수
def classify_motion(keypoints):
    #--- 2024-12-07 추가 3D 모델로 변경
    keypoints = np.array(keypoints).flatten()
    x1, y1, z1 = keypoints[0], keypoints[1], keypoints[2]

    temp = []
    for i in range(0, len(keypoints), 3):
        x, y, z = keypoints[i], keypoints[i+1], keypoints[i+2]
        temp.append(x - x1)
        temp.append(y - y1)
        temp.append(z - z1)

    grouped_keypoints = [temp[i:i+3] for i in range(0, len(temp), 3)]
    keypoints = np.array(grouped_keypoints)
    keypoints = np.expand_dims(keypoints, axis=0)
    #--- 2024-12-07 추가 3D 모델로 변경
    #keypoints = keypoints.reshape(1, -1)  # 모델에 입력할 형태로 변경
    predictions = model.predict(keypoints)  # 모델 예측
    probabilities = predictions[0]  # 각 클래스별 확률
    class_id = np.argmax(probabilities)  # 가장 높은 확률의 클래스 선택
    class_name = class_mapping[class_id]
    return class_name

def process(frame):
    # 프레임 처리
    keypoints, landmarks = extract_keypoints(frame)
    if keypoints is not None:
        # 주요 Keypoint 그리기
        draw_keypoints(frame, keypoints)

        # 스켈레톤 그리기
        draw_skeleton(frame, landmarks)

        # 모션 분류 및 확률 계산
        action = classify_motion(keypoints)

        # 프레임에 텍스트로 결과 출력
        cv2.putText(frame, f"Action: {action}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255, 0, 0), 2)
    else:
        action = "None"

    return frame, action

def save_video(frames, fps):
    # 현재 시간을 파일 이름으로 사용
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    video_filename = f"/content/drive/MyDrive/FallDetectionAVI/{current_time}.avi"

    # 비디오 파일 저장 설정
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    # 프레임을 비디오 파일에 쓰기
    for frame in frames:
        out.write(frame)
    out.release()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 54 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from flask import request, Response
import threading
import os
import cv2
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=25)  # 최대 작업 스레드
frame_buffer = [] # 프레임 버퍼
target_frame = 15
frame_buffer_size = target_frame*5

fall_count = 0
thread_lock = threading.Lock()

def process_frame_logic(file):
    global fall_count
    global frame_buffer
    try:
        # 이미지를 OpenCV 형식으로 변환
        file_bytes = np.frombuffer(file.read(), np.uint8)
        frame = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

        # 프레임 버퍼에 현재 프레임 저장
        with thread_lock:
            frame_buffer.append(frame)
            if len(frame_buffer) > frame_buffer_size:
                frame_buffer.pop(0)

        # 사용자 정의 처리 함수 호출
        frame, action = process(frame)

        # OpenCV 프레임을 JPEG로 인코딩
        _, buffer = cv2.imencode('.jpg', frame)

        if action == "fall":
            with thread_lock:
                fall_count += 1
                if fall_count > target_frame*3:
                    return Response(buffer.tobytes(), mimetype='image/jpeg'), 201
                elif fall_count < target_frame:
                    save_video(frame_buffer, target_frame)
                return Response(buffer.tobytes(), mimetype='image/jpeg'), 200
        else:
            if action != "None":
                with thread_lock:
                    fall_count = 0
            return Response(buffer.tobytes(), mimetype='image/jpeg'), 200
    except Exception as e:
        return {"error": f"Server encountered an error: {str(e)}"}, 500



In [ ]:
from flask import Flask
import threading
import os

app = Flask(__name__)

@app.route('/process_frame', methods=['POST'])
def process_frame():
    if 'file' not in request.files:
        return {"error": "No file part in the request."}, 400

    file = request.files['file']

    # 스레드풀에서 비동기 작업 실행
    future = executor.submit(process_frame_logic, file)
    response, status_code = future.result()  # 작업 결과를 기다림
    return response, status_code

# Flask 서버 실행
def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True)

# ngrok 실행
def run_ngrok():
    os.system("ngrok http 5000")

# Flask와 ngrok를 각각 다른 쓰레드로 실행
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

ngrok_thread = threading.Thread(target=run_ngrok)
ngrok_thread.daemon = True
ngrok_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!./ngrok http --log=stdout 5000

INFO[12-19|05:29:21] no configuration paths supplied 
INFO[12-19|05:29:21] using configuration at default config path path=/root/.config/ngrok/ngrok.yml
INFO[12-19|05:29:21] open config file                         path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-12-19T05:29:21+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2024-12-19T05:29:21+0000 lvl=info msg="client session established" obj=tunnels.session
t=2024-12-19T05:29:21+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-12-19T05:29:21+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:5000 url=https://87e3-34-143-245-43.ngrok-free.app


INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:30:53] "POST /process_frame HTTP/1.1" 400 -


t=2024-12-19T05:30:53+0000 lvl=info msg="join connections" obj=join id=82d41b808dde l=127.0.0.1:5000 r=61.99.5.146:63580
t=2024-12-19T05:30:57+0000 lvl=info msg="join connections" obj=join id=455f4945bf86 l=127.0.0.1:5000 r=61.99.5.146:63582
t=2024-12-19T05:30:57+0000 lvl=info msg="join connections" obj=join id=4cb3215a2452 l=127.0.0.1:5000 r=61.99.5.146:63581
t=2024-12-19T05:30:57+0000 lvl=info msg="join connections" obj=join id=e27cddad63bc l=127.0.0.1:5000 r=61.99.5.146:63583
t=2024-12-19T05:30:57+0000 lvl=info msg="join connections" obj=join id=a9027aef9ce3 l=127.0.0.1:5000 r=61.99.5.146:63584


INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:30:58] "POST /process_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:30:58] "POST /process_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:30:58] "POST /process_frame HTTP/1.1" 200 -


t=2024-12-19T05:30:58+0000 lvl=info msg="join connections" obj=join id=46a10023f960 l=127.0.0.1:5000 r=61.99.5.146:63585


...

INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:54:40] "POST /process_frame HTTP/1.1" 201 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:54:41] "POST /process_frame HTTP/1.1" 201 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 05:54:41] "POST /process_frame HTTP/1.1" 201 -
